In [3]:
import threading
import numpy as np
import time
import cv2
import pandas as pd
from skimage.color import rgb2gray

from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model

import matplotlib.pyplot as plt

model = load_model('restmodel.h5')
print("Loaded model from disk")

from keras.preprocessing import image
labels = {0: 'cardboard', 1: 'glass', 2: 'metal', 3: 'paper', 4: 'plastic', 5: 'trash'}
def imagebox(per,gray):
    wx=1000
    wy=0
    hx=1000
    hy=0
    for i in range(0,gray.shape[0]):
        for j in range(1,gray.shape[1]):
            if (200*max((gray[i][j-1]-gray[i][j])/(gray[i][j-1]+gray[i][j]),-1*(gray[i][j-1]-gray[i][j])/(gray[i][j-1]+gray[i][j])))>per:
                wx=min(wx,j)
                
            if (200*max((gray[i][j-1]-gray[i][j])/(gray[i][j-1]+gray[i][j]),-1*(gray[i][j-1]-gray[i][j])/(gray[i][j-1]+gray[i][j])))>per:
                wy=max(wy,j)
    for i in range(0,gray.shape[1]):
        for j in range(1,gray.shape[0]):
            if (200*max((gray[j-1][i]-gray[j][i])/(gray[j-1][i]+gray[j][i]),-1*(gray[j-1][i]-gray[j][i])/(gray[j-1][i]+gray[j][i])))>per:
                hx=min(hx,j)  
            if (200*max((gray[j-1][i]-gray[j][i])/(gray[j-1][i]+gray[j][i]),-1*(gray[j-1][i]-gray[j][i])/(gray[j-1][i]+gray[j][i])))>per:
                hy=max(hy,j)  
                
   
            
                
    return [wx/gray.shape[1],wy/gray.shape[1],hx/gray.shape[0],hy/gray.shape[0]]

window_name = 'Image'
  
font = cv2.FONT_HERSHEY_SIMPLEX 
  

org = (30, 30) 

fontScale = 0.50
   
color = (255, 255, 255) 
 
thickness = 1


def frd():
    global k
    k=[[8.95389066e-17, 7.45592024e-06, 1.24596785e-17, 9.99992490e-01,3.87652665e-13, 2.87977697e-11]]
    time.sleep(10)
    while(True):
        
        ans=frame1
        ans= cv2.resize(ans, (512,384)) 
        ans = image.img_to_array(ans, dtype=np.uint8)
        ans=np.array(ans)/255
        k=model.predict(ans[np.newaxis, ...])
        if (j==1):
            break
    
def pas():
    global frame1
    frame1=[[[23,23,23]]]
    global j
    
    
    cap = cv2.VideoCapture(0)
    
    j=0
    pre=''
    while(True):
        
        start=time.time()
        ret, frame = cap.read()
        frame=cv2.flip(frame, 1)
        
        
        img=rgb2gray(frame)
        img = cv2.resize(img, (40,40)) 
        l=imagebox(25,img)
        l[0]*=frame.shape[1]
        l[1]*=frame.shape[1]
        l[2]*=frame.shape[0]
        l[3]*=frame.shape[0]
        left=int(l[0])-5
        right=int(l[1])+5
        top=int(l[2])-5
        down=int(l[3])+5
        l.clear()
        
        frame[top-3:top,left-3:right+3]=1
        frame[down-3:down,left-3:right+3]=1
        frame[top:down,left-3:left]=1
        frame[top:down,right:right+3]=1
        if(top<down and left <right):
            frame1=frame[top:down,left:right]
        end=time.time()
        frame = cv2.putText(frame,labels[np.argmax(k[0], axis=-1)]+"/"+str(int(1/(end-start)))+"/"+str(np.max(k[0], axis=-1)),(50, 50),cv2.FONT_HERSHEY_SIMPLEX, 0.5,(255, 0, 0), 2)
        cv2.imshow('frame',frame)
   
        if cv2.waitKey(1) & 0xFF == ord('q'):
            j=1
            break
    cap.release()
    cv2.destroyAllWindows()
t2 = threading.Thread(target=pas) 
t1 = threading.Thread(target=frd) 
 
    # starting thread 2 
t1.start() 
    # starting thread 2 
t2.start() 


    # wait until thread 1 is completely executed 
t1.join() 
    # wait until thread 2 is completely executed 
t2.join() 


Loaded model from disk
